<a href="https://colab.research.google.com/github/Sungi-Hwang/Carclassification/blob/main/Eff8_HSG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, random, unicodedata, numpy as np, torch, gc
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from torchvision.models import efficientnet_b7, EfficientNet_B7_Weights
from torchvision.datasets.folder import IMG_EXTENSIONS
from PIL import Image, ImageFile; ImageFile.LOAD_TRUNCATED_IMAGES = True
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from tqdm import tqdm

In [ ]:
# # 구글 드라이브 마운트
# from google.colab import drive
# drive.mount('/content/drive')

# # 압축 해제
# !unzip -oq "/content/drive/MyDrive/Colab Notebooks/open.zip" -d /content/

In [ ]:
# ─── 1. CFG & Seed ──────────────────────────────────────────────
CFG = dict(IMG_SIZE=600, BATCH_SIZE=128, EPOCHS=10,
           LEARNING_RATE=1e-4, SEED=42,
           NUM_W=min(os.cpu_count(), 32))
def seed_all(s):
    random.seed(s); np.random.seed(s)
    torch.manual_seed(s); torch.cuda.manual_seed(s)
seed_all(CFG['SEED'])
# ─── 2. Dataset ─────────────────────────────────────────────────
MERGE_MAP = {'K5_하이브리드_3세대_2020_2023':'K5_3세대_하이브리드_2020_2022',
             '디_올_뉴_니로_2022_2025':'디_올뉴니로_2022_2025',
             '박스터_718_2017_2024':'718_박스터_2017_2024'}
MERGE_MAP = {unicodedata.normalize("NFC",k):
             unicodedata.normalize("NFC",v) for k,v in MERGE_MAP.items()}

class CustomImageDataset(Dataset):
    def __init__(self, root, transform):
        self.transform, self.samples = transform, []
        cls_dirs = [d for d in os.scandir(root) if d.is_dir()]
        cls_dirs = [unicodedata.normalize("NFC",d.name) for d in cls_dirs]
        unified  = {c: MERGE_MAP.get(c,c) for c in cls_dirs}
        self.classes = sorted(set(unified.values()))
        self.cls2idx = {c:i for i,c in enumerate(self.classes)}
        for orig in cls_dirs:
            lbl = self.cls2idx[unified[orig]]
            for f in os.scandir(os.path.join(root,orig)):
                if f.is_file() and f.name.lower().endswith(tuple(IMG_EXTENSIONS)):
                    self.samples.append((f.path,lbl))
    def __len__(self): return len(self.samples)
    def __getitem__(self, i):
        p,l = self.samples[i]
        img  = self.transform(Image.open(p).convert('RGB'))
        return img,l


In [ ]:
from torchvision.models import EfficientNet_B7_Weights
from torchvision import transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import numpy as np

# 1️⃣  기본 세팅 ----------------------------------------------------
weights   = EfficientNet_B7_Weights.DEFAULT        # == IMAGENET1K_V1
base_tf   = weights.transforms()                   # Resize → CenterCrop → ToTensor → Normalize

IMG_SIZE  = CFG['IMG_SIZE']                        # ex) 600 for B7
BATCH_SZ  = CFG['BATCH_SIZE']

# 2️⃣  Train / Val 전처리 -------------------------------------------
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.7, 1.0), antialias=True),
    transforms.RandomHorizontalFlip(),
    *base_tf.transforms                           # 표준화까지 한꺼번에
])

val_tf = transforms.Compose([
    transforms.Resize(int(IMG_SIZE * 1.15), antialias=True),
    transforms.CenterCrop(IMG_SIZE),
    *base_tf.transforms
])

# 3️⃣  데이터셋 & 로더 -----------------------------------------------
root = "/content/train"
ds_full   = CustomImageDataset(root, transform=train_tf)
ds_full_v = CustomImageDataset(root, transform=val_tf)   # 샘플 순서 동일, TF만 다름

targets = [lbl for _, lbl in ds_full.samples]
tr_idx, v_idx = train_test_split(
    np.arange(len(ds_full)),
    test_size=0.20,
    stratify=targets,
    random_state=42
)

train_loader = DataLoader(
    Subset(ds_full, tr_idx),
    batch_size=BATCH_SZ,
    shuffle=True,
    num_workers=CFG['NUM_W'],
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=4
)

val_loader = DataLoader(
    Subset(ds_full_v, v_idx),
    batch_size=BATCH_SZ,
    shuffle=False,
    num_workers=max(1, CFG['NUM_W'] // 2),
    pin_memory=True,
    persistent_workers=True
)


AttributeError: 'ImageClassification' object has no attribute 'transforms'

In [ ]:
# ─── 4. Model ───────────────────────────────────────────────────
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32      = True
AMP = torch.bfloat16
device = torch.device('cuda')

class BaseModel(nn.Module):
    def __init__(self,nc):
        super().__init__()
        self.backbone = efficientnet_b7(weights=w)
        in_f = self.backbone.classifier[1].in_features
        self.backbone.classifier[1] = nn.Linear(in_f, nc)
    def forward(self,x): return self.backbone(x)

model = BaseModel(len(ds_full.classes)).to(device,
        memory_format=torch.channels_last, dtype=AMP)
model = torch.compile(model, mode='max-autotune')

opt = torch.optim.AdamW(model.parameters(),
        lr=CFG['LEARNING_RATE'], fused=True)
crit = nn.CrossEntropyLoss(); best = np.inf

In [ ]:
# ─── 5. Train / Val ─────────────────────────────────────────────
for ep in range(CFG['EPOCHS']):
    # train
    model.train(); tl=0
    for x,y in tqdm(train_loader, desc=f"Ep{ep+1} Train"):
        x = x.to(device, memory_format=torch.channels_last); y=y.to(device)
        opt.zero_grad(set_to_none=True)
        with autocast(dtype=AMP):
            out = model(x); loss = crit(out,y)
        loss.backward(); opt.step(); tl+=loss.item()
    # val
    model.eval(); all_p, all_y = [], []; correct=total=0
    with torch.no_grad():
        for x,y in tqdm(val_loader, desc=f"Ep{ep+1} Val"):
            x=x.to(device, memory_format=torch.channels_last); y=y.to(device)
            with autocast(dtype=AMP):
                out=model(x); loss=crit(out,y)
            prob = F.softmax(out,1)
            correct += (prob.argmax(1)==y).sum().item(); total+=y.size(0)
            all_p.append(prob.cpu()); all_y.append(y.cpu())
    logloss = log_loss(torch.cat(all_y), torch.cat(all_p))
    acc = 100*correct/total
    print(f"Ep{ep+1}  logloss={logloss:.4f}  acc={acc:.2f}%")
    if logloss < best:
        best = logloss
        torch.save({k:v.cpu() for k,v in model.state_dict().items()},
                   'best_model.pth')
        print("  ✅ best_model.pth saved")